In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist, cifar10

# Task 1: Image Compression


def compress_image(input_path, output_path, quality, format):
    # Read the image
    img = cv2.imread(input_path)
    if img is None:
        print("Error: Could not load image!")
        return

    img = np.float32(img)  # Keep pixel range in [0, 255] but float for DCT

    if format == 'JPEG':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)  # Convert to YCrCb
        Y, Cr, Cb = cv2.split(img)  # Separate channels
    else:
        # PNG (Apply DCT separately on R, G, and B channels)
        Y, Cr, Cb = cv2.split(img)

    # Define JPEG Quantization Matrix (scaled based on quality)
    Q = np.array([
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99]
    ]) * (100 - quality) / 50  # Scale based on quality

    def dct_compress_channel(channel):
        """ Applies DCT compression block-wise on a single channel """
        h, w = channel.shape
        h_pad = 8 - (h % 8) if h % 8 != 0 else 0
        w_pad = 8 - (w % 8) if w % 8 != 0 else 0

        channel_padded = np.pad(channel, ((0, h_pad), (0, w_pad)), mode='constant')
        h_padded, w_padded = channel_padded.shape
        compressed_channel = np.zeros_like(channel_padded)

        for y in range(0, h_padded, 8):
            for x in range(0, w_padded, 8):
                block = channel_padded[y:y+8, x:x+8] - 128  # Shift for DCT
                dct_block = cv2.dct(block)
                quantized_block = np.round(dct_block / Q) * Q
                idct_block = cv2.idct(quantized_block) + 128  # Shift back
                compressed_channel[y:y+8, x:x+8] = idct_block

        return compressed_channel[:h, :w]  # Remove padding

    # Apply DCT compression
    Y_compressed = dct_compress_channel(Y)
    if format == 'JPEG':
        # JPEG: Keep Cr and Cb unchanged
        img_compressed = cv2.merge([Y_compressed, Cr, Cb])
        img_compressed = cv2.cvtColor(img_compressed, cv2.COLOR_YCrCb2BGR)  # Convert back to BGR
    else:
        # PNG: Apply DCT to all channels
        Cr_compressed = dct_compress_channel(Cr)
        Cb_compressed = dct_compress_channel(Cb)
        img_compressed = cv2.merge([Y_compressed, Cr_compressed, Cb_compressed])

    # Normalize back to 0-255
    img_compressed = np.clip(img_compressed, 0, 255).astype(np.uint8)

    # Save compressed image
    if format == 'JPEG':
        cv2.imwrite(output_path, img_compressed, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    elif format == 'PNG':
        cv2.imwrite(output_path, img_compressed, [int(cv2.IMWRITE_PNG_COMPRESSION), 9 - quality // 10])

    print(f"DCT compressed {format} image saved at {output_path}")

input_image="test.jpg"
op_path_jpeg="compressed_img.jpg"
op_path_png="compressed_img.png"
jpeg_quality=50
png_compression=50

compress_image(input_image, op_path_jpeg, jpeg_quality, "JPEG")

compress_image(input_image, op_path_png, png_compression, "PNG")

DCT compressed JPEG image saved at compressed_img.jpg
DCT compressed PNG image saved at compressed_img.png


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocessing MNIST
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Build CNN Model for MNIST
def build_mnist_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train MNIST Model
mnist_model = build_mnist_cnn()
mnist_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate MNIST Model
y_pred_mnist = mnist_model.predict(x_test)
y_pred_classes_mnist = np.argmax(y_pred_mnist, axis=1)
y_true_mnist = np.argmax(y_test, axis=1)
print("MNIST Classification Report:")
print(classification_report(y_true_mnist, y_pred_classes_mnist))
print("Confusion Matrix:")
print(confusion_matrix(y_true_mnist, y_pred_classes_mnist))

# Load CIFAR-10 dataset
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocessing CIFAR-10
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Build CNN Model for CIFAR-10
def build_cifar10_cnn():
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train CIFAR-10 Model
cifar10_model = build_cifar10_cnn()
cifar10_model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate CIFAR-10 Model
y_pred_cifar10 = cifar10_model.predict(x_test)
y_pred_classes_cifar10 = np.argmax(y_pred_cifar10, axis=1)
y_true_cifar10 = np.argmax(y_test, axis=1)
print("CIFAR-10 Classification Report:")
print(classification_report(y_true_cifar10, y_pred_classes_cifar10))
print("Confusion Matrix:")
print(confusion_matrix(y_true_cifar10, y_pred_classes_cifar10))


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 48s 62ms/step - accuracy: 0.8750 - loss: 0.4209 - val_accuracy: 0.9808 - val_loss: 0.0656
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 59ms/step - accuracy: 0.9808 - loss: 0.0624 - val_accuracy: 0.9840 - val_loss: 0.0530
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 80s 57ms/step - accuracy: 0.9887 - loss: 0.0362 - val_accuracy: 0.9881 - val_loss: 0.0410
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 81s 56ms/step - accuracy: 0.9918 - loss: 0.0258 - val_accuracy: 0.9884 - val_loss: 0.0391
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 55ms/step - accuracy: 0.9928 - loss: 0.0216 - val_accuracy: 0.9886 - val_loss: 0.0395
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 82s 56ms/step - accuracy: 0.9947 - loss: 0.0170 - val_accuracy: 0.9875 - val_loss: 0.0426
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 83s 58ms/step - accuracy: 0.9967 - loss: 0.0105 - val_accuracy: 0.9898 - val_loss: 0.0360
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━